# TCGA-LUSC CT Scan Analysis with TotalSegmentator

This notebook demonstrates how to:
1. Load DICOM files from TCGA dataset
2. Build 3D volumes from CT scans
3. Visualize CT data
4. Perform automatic lung tumor segmentation using TotalSegmentator
5. Extract radiomic features using PyRadiomics

## Step 0: Setup Python 3.10 Virtual Environment

**This cell sets up the entire environment. Run it first!**

In [ ]:
# Install Python 3.10 and create virtual environment
!sudo apt-get update -qq
!sudo apt-get install -y python3.10 python3.10-venv python3.10-distutils git build-essential

# Create virtual environment
!python3.10 -m venv /content/py310env

# Upgrade pip and install all required packages
!/content/py310env/bin/pip install --upgrade pip
!/content/py310env/bin/pip install -q pydicom numpy matplotlib ipywidgets SimpleITK tqdm PyWavelets pylibjpeg

# ================================
# Install PyRadiomics FROM SOURCE
# (Because pip version does NOT support Python 3.10)
# ================================
!git clone https://github.com/Radiomics/pyradiomics.git /content/pyradiomics
%cd /content/pyradiomics

# Install PyRadiomics requirements
!/content/py310env/bin/pip install -r requirements.txt

# Build PyRadiomics C extensions & install
!/content/py310env/bin/python setup.py build_ext --inplace
!/content/py310env/bin/python setup.py install

# ================================
# Install TotalSegmentator (same as before)
# ================================
!/content/py310env/bin/pip install -q git+https://github.com/wasserth/TotalSegmentator.git

# Verify installation
print("\n=== Verification ===")
!/content/py310env/bin/python --version
!/content/py310env/bin/pip list | grep -E "pydicom|pyradiomics|SimpleITK|TotalSegmentator"
print("\n✓ Environment setup complete!")

## Step 1: Mount Google Drive and Find DICOM Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Path to your TCGA dataset
DATASET_PATH = "/content/drive/MyDrive/nd/manifest-27oBhciT6753657225075401932/TCGA-LUSC"

def find_dicom_files(root):
    """Recursively find all DICOM files in directory"""
    dicom_paths = []
    for dirpath, _, filenames in os.walk(root):
        for f in filenames:
            if f.lower().endswith(".dcm"):
                dicom_paths.append(os.path.join(dirpath, f))
    return dicom_paths

dicoms = find_dicom_files(DATASET_PATH)
print(f"Number of DICOM files found: {len(dicoms)}")
if dicoms:
    print(f"Example DICOM: {dicoms[0]}")

## Step 2: Load DICOM Series and Build 3D Volume

In [ ]:
# Save the DICOM loading script
script_content = """
import pydicom
import numpy as np
import os
import pickle

def load_dicom_series(folder):
    slices = []
    for f in sorted(os.listdir(folder)):
        if f.lower().endswith(".dcm"):
            path = os.path.join(folder, f)
            try:
                ds = pydicom.dcmread(path)
                if hasattr(ds, 'PixelData'):
                    slices.append(ds)
            except Exception as e:
                print(f"Warning: Could not read {f}: {e}")
                continue

    if len(slices) == 0:
        raise ValueError(f"No valid DICOM slices found in folder: {folder}")

    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    volume = np.stack([s.pixel_array for s in slices]).astype(np.int16)

    for i, s in enumerate(slices):
        intercept = float(s.RescaleIntercept)
        slope = float(s.RescaleSlope)
        volume[i] = volume[i] * slope + intercept

    px_spacing = list(map(float, slices[0].PixelSpacing))

    try:
        z_spacing = abs(
            float(slices[1].ImagePositionPatient[2]) -
            float(slices[0].ImagePositionPatient[2])
        )
    except:
        z_spacing = float(slices[0].SliceThickness)

    spacing = (z_spacing, px_spacing[0], px_spacing[1])
    return volume, spacing, slices

patient_folder = "/content/drive/MyDrive/nd/manifest-27oBhciT6753657225075401932/TCGA-LUSC/TCGA-34-2605/08-10-1989-NA-CT CHEST-64467/2.000000-Helical 5s-02217"

print("Loading DICOM series...")
volume, spacing, dicom_slices = load_dicom_series(patient_folder)

print(f"Volume shape: {volume.shape}")
print(f"Number of slices: {len(dicom_slices)}")
print(f"Spacing (z, y, x): {spacing}")
print(f"HU range: [{volume.min()}, {volume.max()}]")

# Save results for use in main notebook
with open('/content/dicom_data.pkl', 'wb') as f:
    pickle.dump({
        'volume': volume,
        'spacing': spacing,
        'num_slices': len(dicom_slices)
    }, f)
"""

with open('/content/load_dicom.py', 'w') as f:
    f.write(script_content)

# Run with Python 3.10 venv
!/content/py310env/bin/python /content/load_dicom.py

# Load results into main notebook
import pickle
with open('/content/dicom_data.pkl', 'rb') as f:
    data = pickle.load(f)
    volume = data['volume']
    spacing = data['spacing']
    num_slices = data['num_slices']

print("\n✓ Data loaded into notebook variables")

## Step 3: Visualize CT Scans

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def window_image(img, window_center=-600, window_width=1500):
    """Apply windowing for better visualization"""
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def show_slice(volume, slice_index=None, window=True):
    """Display a single slice"""
    if slice_index is None:
        slice_index = volume.shape[0] // 2

    img = volume[slice_index]
    if window:
        img = window_image(img)

    plt.figure(figsize=(6, 6))
    plt.imshow(img, cmap="gray")
    plt.title(f"Slice {slice_index} (Lung Window)")
    plt.axis("off")
    plt.show()

show_slice(volume)

In [ ]:
# Interactive slice viewer
from ipywidgets import interact, IntSlider

def scroll_slices(index):
    plt.figure(figsize=(6, 6))
    plt.imshow(window_image(volume[index]), cmap="gray")
    plt.title(f"Slice {index}/{volume.shape[0]-1}")
    plt.axis("off")
    plt.show()

interact(scroll_slices,
         index=IntSlider(min=0, max=volume.shape[0]-1, step=1, value=volume.shape[0]//2))

In [ ]:
# Maximum Intensity Projection (MIP)
mip = np.max(volume, axis=0)

plt.figure(figsize=(6, 6))
plt.imshow(mip, cmap="gray")
plt.title("Maximum Intensity Projection (MIP)")
plt.axis("off")
plt.show()

## Step 4: Save Volume as NIfTI for TotalSegmentator

In [ ]:
# Save NIfTI using Python 3.10 environment
save_nifti_script = f"""
import SimpleITK as sitk
import pickle

# Load data
with open('/content/dicom_data.pkl', 'rb') as f:
    data = pickle.load(f)

volume = data['volume']
spacing = data['spacing']

# Convert to SimpleITK image
sitk_img = sitk.GetImageFromArray(volume)
sitk_img.SetSpacing(spacing[::-1])  # SimpleITK uses (x,y,z) order

# Save as NIfTI
output_path = "/content/ct_volume.nii.gz"
sitk.WriteImage(sitk_img, output_path)

print(f"✓ Saved NIfTI file: {{output_path}}")
print(f"  Image size: {{sitk_img.GetSize()}}")
print(f"  Spacing: {{sitk_img.GetSpacing()}}")
"""

with open('/content/save_nifti.py', 'w') as f:
    f.write(save_nifti_script)

!/content/py310env/bin/python /content/save_nifti.py

## Step 5: Run TotalSegmentator for Lung Tumor Detection

In [ ]:
# Run TotalSegmentator using Python 3.10 environment
print("Running TotalSegmentator...")
print("This may take several minutes...\n")

!/content/py310env/bin/TotalSegmentator -i /content/ct_volume.nii.gz -o /content/ts_output --fast

print("\n✓ Segmentation complete!")

## Step 6: Load and Visualize Tumor Segmentation

In [ ]:
# Check what organs were segmented
import os

output_dir = "/content/ts_output"
if os.path.exists(output_dir):
    segmented_organs = [f.replace('.nii.gz', '') for f in os.listdir(output_dir)
                       if f.endswith('.nii.gz')]
    print("Segmented structures:")
    for organ in sorted(segmented_organs):
        print(f"  - {organ}")
else:
    print("Output directory not found!")

In [ ]:
# Load segmentation mask using Python 3.10
load_mask_script = """
import SimpleITK as sitk
import numpy as np
import pickle
import os

mask_path = "/content/ts_output/lung_tumor.nii.gz"
mask = None

if os.path.exists(mask_path):
    mask_img = sitk.ReadImage(mask_path)
    mask = sitk.GetArrayFromImage(mask_img)
    print(f"Tumor mask shape: {mask.shape}")
    print(f"Tumor voxels detected: {np.sum(mask > 0)}")
else:
    print("⚠️  Warning: lung_tumor.nii.gz not found!")
    print("   Trying to load lung lobes instead...")

    lung_paths = ['/content/ts_output/lung_upper_lobe_left.nii.gz',
                  '/content/ts_output/lung_upper_lobe_right.nii.gz']

    for lp in lung_paths:
        if os.path.exists(lp):
            mask_img = sitk.ReadImage(lp)
            mask = sitk.GetArrayFromImage(mask_img)
            print(f"\nLoaded {os.path.basename(lp)} instead")
            break

if mask is not None:
    with open('/content/mask_data.pkl', 'wb') as f:
        pickle.dump({'mask': mask}, f)
"""

"""with open('/content/load_mask.py', 'w') as f:
    f.write(load_mask_script)"""

!/content/py310env/bin/python /content/load_mask.py

# Load mask into main notebook
import pickle
with open('/content/mask_data.pkl', 'rb') as f:
    mask_data = pickle.load(f)
    mask = mask_data['mask']

print("\n✓ Mask loaded into notebook")

In [ ]:
# Visualize segmentation overlay
def show_overlay(volume, mask, slice_index=None, alpha=0.5):
    """Show CT scan with segmentation overlay"""
    if slice_index is None:
        slice_index = volume.shape[0] // 2

    plt.figure(figsize=(8, 8))
    plt.imshow(window_image(volume[slice_index]), cmap="gray")

    if np.any(mask[slice_index] > 0):
        plt.imshow(mask[slice_index], cmap="Reds", alpha=alpha)
        plt.title(f"Slice {slice_index} - Segmentation Overlay")
    else:
        plt.title(f"Slice {slice_index} - No segmentation on this slice")

    plt.axis("off")
    plt.colorbar(label="Segmentation")
    plt.show()

# Find slice with maximum segmentation
slice_sums = np.sum(mask, axis=(1, 2))
best_slice = np.argmax(slice_sums)
print(f"Slice with most segmentation: {best_slice}\n")
show_overlay(volume, mask, best_slice)

In [ ]:
# Interactive overlay viewer
from ipywidgets import interact, IntSlider, FloatSlider

def scroll_overlay(index, alpha=0.5):
    show_overlay(volume, mask, index, alpha)

interact(scroll_overlay,
         index=IntSlider(min=0, max=volume.shape[0]-1, value=best_slice, description='Slice'),
         alpha=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='Opacity'))

## Step 7: Extract Radiomic Features with PyRadiomics

In [ ]:
# ================================
# Install PyRadiomics FROM SOURCE (new pyproject.toml build system)
# ================================
!git clone https://github.com/Radiomics/pyradiomics.git /content/pyradiomics
%cd /content/pyradiomics

# Build & install using pip (this replaces setup.py)
!/content/py310env/bin/pip install .

In [ ]:
# Extract radiomic features using Python 3.10 environment
radiomics_script = """
import SimpleITK as sitk
from radiomics import featureextractor
import json

print("Extracting radiomic features...")

# Initialize feature extractor
extractor = featureextractor.RadiomicsFeatureExtractor()

# Load image and mask
image_path = "/content/ct_volume.nii.gz"
mask_path = "/content/ts_output/lung_tumor.nii.gz"

# Check if tumor mask exists, otherwise use lung lobe
import os
if not os.path.exists(mask_path):
    mask_path = "/content/ts_output/lung_upper_lobe_left.nii.gz"
    print(f"Using {mask_path} instead of tumor mask")

# Extract features
features = extractor.execute(image_path, mask_path)

# Filter out metadata, keep only feature values
feature_dict = {}
for key, val in features.items():
    if not key.startswith('diagnostics_'):
        feature_dict[key] = float(val) if hasattr(val, '__float__') else str(val)

# Save to JSON
with open('/content/radiomic_features.json', 'w') as f:
    json.dump(feature_dict, f, indent=2)

print(f"\n✓ Extracted {len(feature_dict)} radiomic features")
print("\nSample features:")
for i, (k, v) in enumerate(list(feature_dict.items())[:5]):
    print(f"  {k}: {v}")
"""

"""with open('/content/extract_radiomics.py', 'w') as f:
    f.write(radiomics_script)"""
!/content/py310env/bin/python /content/extract_radiomics.py

In [ ]:
# Load and display radiomic features
import json
import pandas as pd

with open('/content/radiomic_features.json', 'r') as f:
    features = json.load(f)

# Convert to DataFrame for better visualization
df = pd.DataFrame(list(features.items()), columns=['Feature', 'Value'])

print(f"Total features extracted: {len(df)}\n")
print("Feature categories:")
categories = df['Feature'].str.split('_').str[0].value_counts()
print(categories)

print("\nFirst 10 features:")
print(df.head(10).to_string(index=False))

## Summary

This notebook demonstrated:
1. ✓ Setting up Python 3.10 virtual environment in Colab
2. ✓ Loading DICOM files from TCGA-LUSC dataset
3. ✓ Building 3D volumes with proper spacing
4. ✓ Visualizing CT scans with windowing
5. ✓ Running TotalSegmentator for automatic segmentation
6. ✓ Visualizing segmentation results
7. ✓ Extracting radiomic features with PyRadiomics

**Note**: All processing that requires PyRadiomics or specific Python versions is handled through the Python 3.10 virtual environment, while visualization and basic operations use the default Colab environment.

In [ ]:
import os

base_folder = "/content/drive/MyDrive/nd/manifest-27oBhciT6753657225075401932/TCGA-LUSC"

patients = [d for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d))]

print("Number of patient folders:", len(patients))
print("\nFirst 10 patient IDs:")
print(patients[:10])

In [ ]:
import os

base_folder = "/content/drive/MyDrive/nd/manifest-27oBhciT6753657225075401932/TCGA-LUSC"

def find_dicom_series(patient_path):
    """Return the deepest folder that contains DICOM files."""
    for root, dirs, files in os.walk(patient_path):
        dcm_files = [f for f in files if f.lower().endswith(".dcm")]
        if len(dcm_files) > 5:  # found a CT series
            return root
    return None

patient_ct_paths = {}

for patient in os.listdir(base_folder):
    p_path = os.path.join(base_folder, patient)
    if os.path.isdir(p_path):
        series_path = find_dicom_series(p_path)
        patient_ct_paths[patient] = series_path

# Show summary
valid = {p: path for p, path in patient_ct_paths.items() if path is not None}

print("Patients with usable CT series:", len(valid))
print("\nExample paths:")
for k in list(valid.keys())[:5]:
    print(k, "→", valid[k])


In [ ]:
# Helper Function
load_code = """import os
import pydicom

BASE = "/content/drive/MyDrive/nd"
root_patients = []

# ---------- Step 1: Find the TCGA-LUSC folder automatically ----------
for root, dirs, files in os.walk(BASE):
    for d in dirs:
        if d == "TCGA-LUSC":
            LUSC_PATH = os.path.join(root, d)
            print("Found TCGA-LUSC at:", LUSC_PATH)
            root_patients.append(LUSC_PATH)

if len(root_patients) == 0:
    raise ValueError("TCGA-LUSC folder not found!")

LUSC_PATH = root_patients[0]   # use first match


# ---------- Step 2: Function to check if folder is a CT series ----------
def is_ct_series(folder):
    dicom_files = [f for f in os.listdir(folder) if f.lower().endswith(".dcm")]

    # Must have enough slices
    if len(dicom_files) < 30:
        return False

    try:
        sample = pydicom.dcmread(os.path.join(folder, dicom_files[0]), stop_before_pixels=True)
    except:
        return False

    # Must be CT
    if "CT" not in str(sample.Modality).upper():
        return False

    # Must have slice position metadata
    try:
        _ = float(sample.ImagePositionPatient[2])
    except:
        return False

    return True


# ---------- Step 3: Scan all patients and detect CT series ----------
patients = {}

for patient_id in os.listdir(LUSC_PATH):
    p_path = os.path.join(LUSC_PATH, patient_id)
    if not os.path.isdir(p_path):
        continue

    best_series = None
    best_slice_count = 0

    # go inside date folders
    for date_folder in os.listdir(p_path):
        date_path = os.path.join(p_path, date_folder)
        if not os.path.isdir(date_path):
            continue

        # go inside series folders
        for series_folder in os.listdir(date_path):
            s_path = os.path.join(date_path, series_folder)
            if not os.path.isdir(s_path):
                continue

            dicom_files = [f for f in os.listdir(s_path) if f.endswith(".dcm")]

            if len(dicom_files) < 20:
                continue

            # check if CT
            try:
                ds = pydicom.dcmread(os.path.join(s_path, dicom_files[0]), stop_before_pixels=True)
            except:
                continue

            if ds.Modality != "CT":
                continue

            # the more slices the better
            if len(dicom_files) > best_slice_count:
                best_slice_count = len(dicom_files)
                best_series = s_path

    if best_series:
        patients[patient_id] = best_series
        print(f"✓ {patient_id} → {best_series}  (slices={best_slice_count})")

print("\n--- FINAL RESULT ---")
print("Patients with valid CT:", len(patients))
patients"""

"""with open('/content/load_code.py', 'w') as f:
    f.write(load_code)"""
!/content/py310env/bin/python /content/load_code.py

In [ ]:
load_code2 = """import os
import pydicom

BASE = "/content/drive/MyDrive/nd"
root_patients = []

# ---------- Step 1: Find the TCGA-LUSC folder automatically ----------
for root, dirs, files in os.walk(BASE):
    for d in dirs:
        if d == "TCGA-LUSC":
            LUSC_PATH = os.path.join(root, d)
            print("Found TCGA-LUSC at:", LUSC_PATH)
            root_patients.append(LUSC_PATH)

if len(root_patients) == 0:
    raise ValueError("TCGA-LUSC folder not found!")

LUSC_PATH = root_patients[0]   # use first match


# ---------- Step 2: Function to check if folder is a CT series ----------
def is_ct_series(folder):
    dicom_files = [f for f in os.listdir(folder) if f.lower().endswith(".dcm")]

    # Must have enough slices
    if len(dicom_files) < 30:
        return False

    try:
        sample = pydicom.dcmread(os.path.join(folder, dicom_files[0]), stop_before_pixels=True)
    except:
        return False

    # Must be CT
    if "CT" not in str(sample.Modality).upper():
        return False

    # Must have slice position metadata
    try:
        _ = float(sample.ImagePositionPatient[2])
    except:
        return False

    return True


# ---------- Step 3: Scan all patients and detect CT series ----------
patients = {}

for patient_id in os.listdir(LUSC_PATH):
    p_path = os.path.join(LUSC_PATH, patient_id)
    if not os.path.isdir(p_path):
        continue

    best_series = None
    best_slice_count = 0

    # go inside date folders
    for date_folder in os.listdir(p_path):
        date_path = os.path.join(p_path, date_folder)
        if not os.path.isdir(date_path):
            continue

        # go inside series folders
        for series_folder in os.listdir(date_path):
            s_path = os.path.join(date_path, series_folder)
            if not os.path.isdir(s_path):
                continue

            dicom_files = [f for f in os.listdir(s_path) if f.endswith(".dcm")]

            if len(dicom_files) < 20:
                continue

            # check if CT
            try:
                ds = pydicom.dcmread(os.path.join(s_path, dicom_files[0]), stop_before_pixels=True)
            except:
                continue

            if ds.Modality != "CT":
                continue

            # the more slices the better
            if len(dicom_files) > best_slice_count:
                best_slice_count = len(dicom_files)
                best_series = s_path

    if best_series:
        patients[patient_id] = best_series
        print(f"✓ {patient_id} → {best_series}  (slices={best_slice_count})")

print("\n--- FINAL RESULT ---")
print("Patients with valid CT:", len(patients))
import os
import pydicom
import numpy as np
import SimpleITK as sitk

# patients dict is already created
print("Patients detected:", len(patients))
os.makedirs("/content/ct_volumes", exist_ok=True)

def load_dicom_series(folder):
    files = sorted([os.path.join(folder,f) for f in os.listdir(folder) if f.endswith(".dcm")])

    slices = []
    for f in files:
        try:
            ds = pydicom.dcmread(f)
            slices.append(ds)
        except:
            continue

    # Sort by slice position
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))

    # Build volume
    volume = np.stack([s.pixel_array for s in slices]).astype(np.int16)

    # Convert to HU
    for i, ds in enumerate(slices):
        intercept = getattr(ds, "RescaleIntercept", 0)
        slope = getattr(ds, "RescaleSlope", 1)
        volume[i] = volume[i] * slope + intercept

    # Convert to SimpleITK image
    itk_img = sitk.GetImageFromArray(volume)

    # Set spacing from DICOM metadata
    px_spacing = list(map(float, slices[0].PixelSpacing))
    slice_thickness = abs(float(slices[1].ImagePositionPatient[2]) - float(slices[0].ImagePositionPatient[2]))
    spacing = [slice_thickness] + px_spacing  # (z, y, x)

    itk_img.SetSpacing(spacing)

    return itk_img


# ---------------------------
# Process all patients
# ---------------------------
for pid, path in patients.items():
    print(f"\nProcessing {pid} ...")

    img = load_dicom_series(path)
    out_path = f"/content/ct_volumes/{pid}.nii.gz"

    sitk.WriteImage(img, out_path)
    print(f"✓ Saved: {out_path}")

print("\nAll CT volumes saved!")
"""

"""with open('/content/load_code2.py', 'w') as f:
    f.write(load_code2)"""
!/content/py310env/bin/python /content/load_code2.py

In [ ]:
import os
import subprocess
import time
import torch

# Configuration
input_folder = "/content/ct_volumes"
output_folder = "/content/ts_output"
os.makedirs(output_folder, exist_ok=True)

# Get all files
nii_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".nii.gz")])

# Print all file names for reference
print("="*60)
print("ALL FILES TO PROCESS:")
print("="*60)
for i, f in enumerate(nii_files, 1):
    size_mb = os.path.getsize(os.path.join(input_folder, f)) / (1024*1024)
    print(f"{i:2d}. {f:<30s} ({size_mb:6.1f} MB)")
print("="*60)
print(f"\nTotal: {len(nii_files)} files\n")

# ============================================================
# CHANGE THIS NUMBER TO PROCESS DIFFERENT FILES (1-17)
# ============================================================
FILE_INDEX = 5  # ← CHANGE THIS NUMBER (1 to 17)
# ============================================================

if FILE_INDEX < 1 or FILE_INDEX > len(nii_files):
    print(f"❌ ERROR: FILE_INDEX must be between 1 and {len(nii_files)}")
else:
    # Get the file to process
    f = nii_files[FILE_INDEX - 1]
    pid = f.replace(".nii.gz", "")
    in_path = os.path.join(input_folder, f)
    out_path = os.path.join(output_folder, pid)

    # Clear GPU cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print(f"\n{'='*60}")
    print(f"PROCESSING FILE {FILE_INDEX}/{len(nii_files)}: {pid}")
    print(f"{'='*60}")
    print(f"Input:  {in_path}")
    print(f"Output: {out_path}")
    print(f"Start time: {time.strftime('%H:%M:%S')}\n")

    # Get file size
    size_mb = os.path.getsize(in_path) / (1024*1024)
    print(f"Input file size: {size_mb:.2f} MB")

    # Check if already processed
    if os.path.exists(out_path) and os.listdir(out_path):
        print(f"\n⚠️ Output folder already exists with {len(os.listdir(out_path))} files")
        print("To re-process, delete the output folder first or skip this file")
    else:
        # Build command
        cmd = [
            "/content/py310env/bin/TotalSegmentator",
            "-i", in_path,
            "-o", out_path,
            "--fast",
            "--roi_subset",
            "lung_lower_lobe_left",
            "lung_lower_lobe_right",
            "lung_middle_lobe_right",
            "lung_upper_lobe_left",
            "lung_upper_lobe_right"
        ]

        print("\nSegmenting lung lobes only...")
        print(f"Command: {' '.join(cmd)}\n")

        # Run the command
        result = subprocess.run(cmd)

        print(f"\nEnd time: {time.strftime('%H:%M:%S')}")

        if result.returncode == 0:
            num_files = len([f for f in os.listdir(out_path) if f.endswith('.nii.gz')])
            print(f"\n✅ SUCCESS! Generated {num_files} segmentation files")
            print(f"Output location: {out_path}")
        else:
            print(f"\n❌ FAILED with return code: {result.returncode}")

    print("\n" + "="*60)
    print(f"To process next file, change FILE_INDEX to {FILE_INDEX + 1}")
    print("="*60)

In [ ]:
p = """import os
import SimpleITK as sitk
import numpy as np

ts_root = "/content/drive/MyDrive/dataset_lung/ts_output/F_dataset"
mask_out = "/content/lung_masks"
os.makedirs(mask_out, exist_ok=True)

lung_lobes = [
    "lung_upper_lobe_left.nii.gz",
    "lung_lower_lobe_left.nii.gz",
    "lung_upper_lobe_right.nii.gz",
    "lung_middle_lobe_right.nii.gz",
    "lung_lower_lobe_right.nii.gz"
]

patients = sorted(os.listdir(ts_root))

print(f"Processing {len(patients)} patients...\n")

for pid in patients:
    pid_folder = os.path.join(ts_root, pid)
    if not os.path.isdir(pid_folder):
        continue

    combined = None
    ref_img = None

    for lobe in lung_lobes:
        lobe_path = os.path.join(pid_folder, lobe)
        if not os.path.exists(lobe_path):
            continue

        img = sitk.ReadImage(lobe_path)
        arr = sitk.GetArrayFromImage(img)

        if combined is None:
            combined = arr.copy()
            ref_img = img
        else:
            combined += arr

    if combined is None:
        print(f"❌ No lung masks found for {pid}")
        continue

    combined = (combined > 0).astype(np.uint8)

    out_img = sitk.GetImageFromArray(combined)
    out_img.CopyInformation(ref_img)

    out_path = os.path.join(mask_out, f"{pid}_lungmask.nii.gz")
    sitk.WriteImage(out_img, out_path)

    print(f"✓ Saved lung mask: {pid}")

print("\n🎉 Lung mask generation completed!")"""

"""with open('/content/load_retrieved_data.py', 'w') as f:
    f.write(p)"""
!/content/py310env/bin/python /content/load_retrieved_data.py

In [ ]:
!cp -r /content/lung_masks /content/drive/MyDrive/dataset_lung/

In [ ]:
fix = """import os
import SimpleITK as sitk
import numpy as np

ct_folder = "/content/ct_volumes"
mask_folder = "/content/lung_masks"
fixed_mask_folder = "/content/lung_masks_fixed"

os.makedirs(fixed_mask_folder, exist_ok=True)

mask_files = [f for f in os.listdir(mask_folder) if f.endswith("_lungmask.nii.gz")]

print(f"Fixing {len(mask_files)} lung masks...\n")

for mf in mask_files:
    pid = mf.replace("_lungmask.nii.gz", "")
    ct_path = os.path.join(ct_folder, f"{pid}.nii.gz")
    mask_path = os.path.join(mask_folder, mf)

    if not os.path.exists(ct_path):
        print(f"⚠️ CT missing for {pid}, skipping")
        continue

    # Load CT and mask
    ct_img = sitk.ReadImage(ct_path)
    mask_img = sitk.ReadImage(mask_path)

    mask_arr = sitk.GetArrayFromImage(mask_img)

    # Sanity check
    if np.sum(mask_arr) == 0:
        print(f"❌ Empty mask detected for {pid}, skipping")
        continue

    # Force binary label = 1
    mask_arr = (mask_arr > 0).astype(np.uint8)

    # Create new mask with CT metadata
    fixed_mask = sitk.GetImageFromArray(mask_arr)
    fixed_mask.CopyInformation(ct_img)

    out_path = os.path.join(fixed_mask_folder, f"{pid}_lungmask.nii.gz")
    sitk.WriteImage(fixed_mask, out_path)

    print(f"✓ Fixed mask saved: {pid}")

print("\n✅ All masks fixed and aligned")
"""
"""with open('/content/radiomics_fix_data.py', 'w') as f:
    f.write(fix)"""
!/content/py310env/bin/python /content/radiomics_fix_data.py

In [ ]:
n = """import os
import pandas as pd
from radiomics import featureextractor

ct_folder = "/content/ct_volumes"
mask_folder = "/content/lung_masks_fixed"

extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.disableAllFeatures()

extractor.enableFeatureClassByName("firstorder")
extractor.enableFeatureClassByName("shape")
extractor.enableFeatureClassByName("glcm")
extractor.enableFeatureClassByName("glrlm")
extractor.enableFeatureClassByName("glszm")

results = []

mask_files = sorted([f for f in os.listdir(mask_folder) if f.endswith("_lungmask.nii.gz")])

print(f"Extracting radiomics for {len(mask_files)} patients...\n")

for mf in mask_files:
    pid = mf.replace("_lungmask.nii.gz", "")
    ct_path = os.path.join(ct_folder, f"{pid}.nii.gz")
    mask_path = os.path.join(mask_folder, mf)

    if not os.path.exists(ct_path):
        print(f"⚠️ CT missing for {pid}, skipping")
        continue

    print(f"→ {pid}")

    features = extractor.execute(ct_path, mask_path)

    row = {"PatientID": pid}
    for k, v in features.items():
        if isinstance(v, (int, float)):
            row[k] = v

    results.append(row)

df = pd.DataFrame(results)
csv_path = "/content/lung_radiomics_features.csv"
df.to_csv(csv_path, index=False)

print(f"\n✅ Radiomics saved to: {csv_path}")"""

"""with open('/content/radiomics_new_data.py', 'w') as f:
    f.write(n)"""
!/content/py310env/bin/python /content/radiomics_new_data.py

In [ ]:
import pandas as pd

df = pd.read_csv("/content/lung_radiomics_features.csv")

print("Patients:", df.shape[0])
print("Features:", df.shape[1] - 1)
df.head()

**Clean & Normalize Radiomics Features**

In [ ]:
import pandas as pd

df = pd.read_csv("/content/lung_radiomics_features.csv")
print(df.columns.tolist())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# --------------------------------
# Load data
# --------------------------------
df = pd.read_csv("/content/lung_radiomics_features.csv")
print("Original shape:", df.shape)

# --------------------------------
# Separate PatientID
# --------------------------------
patient_ids = df["PatientID"]
df = df.drop(columns=["PatientID"])

# --------------------------------
# Remove diagnostics (not radiomics)
# --------------------------------
df = df.loc[:, ~df.columns.str.startswith("diagnostics")]

print("After removing diagnostics:", df.shape)

# --------------------------------
# Remove zero-variance features
# --------------------------------
variance = df.var()
df = df.loc[:, variance > 1e-6]

# --------------------------------
# Handle NaN / Inf
# --------------------------------
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.fillna(df.mean())

# --------------------------------
# Z-score normalization
# --------------------------------
scaler = StandardScaler()
scaled = scaler.fit_transform(df)

df_scaled = pd.DataFrame(scaled, columns=df.columns)
df_scaled.insert(0, "PatientID", patient_ids.values)

# --------------------------------
# Save cleaned dataset
# --------------------------------
out_path = "/content/lung_radiomics_clean.csv"
df_scaled.to_csv(out_path, index=False)

print("✅ Clean radiomics saved to:", out_path)
print("Final shape:", df_scaled.shape)


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/lung_radiomics_clean.csv")

# --------------------------------
# Novel composite features
# --------------------------------
df["Shape_Complexity"] = (
    df["original_shape_MajorAxisLength"] /
    (df["original_shape_MinorAxisLength"] + 1e-6)
)

df["Elongation_Flatness_Ratio"] = (
    df["original_shape_Elongation"] /
    (df["original_shape_Flatness"] + 1e-6)
)

df["Normalized_Lung_Size"] = (
    df["original_shape_VoxelVolume"] /
    df["original_shape_MajorAxisLength"]
)

df["Axis_Disparity"] = (
    df["original_shape_MajorAxisLength"] -
    df["original_shape_LeastAxisLength"]
)

# Save novel features
out_path = "/content/lung_radiomics_novel.csv"
df.to_csv(out_path, index=False)

print("🔥 Novel radiomic biomarkers created!")
print("Saved to:", out_path)
print("Final shape:", df.shape)

**PCA + visualization**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Load data
# -------------------------------
df = pd.read_csv("/content/lung_radiomics_novel.csv")

patient_ids = df["PatientID"]
X = df.drop(columns=["PatientID"])

print("Data shape:", X.shape)

In [ ]:
# -------------------------------
# PCA
# -------------------------------
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
pca_df["PatientID"] = patient_ids.values

print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Total variance explained:", pca.explained_variance_ratio_.sum())

In [ ]:
plt.figure(figsize=(7, 6))
plt.scatter(pca_df["PC1"], pca_df["PC2"], s=80)

for i, pid in enumerate(pca_df["PatientID"]):
    plt.text(
        pca_df["PC1"][i] + 0.02,
        pca_df["PC2"][i] + 0.02,
        pid,
        fontsize=9
    )

plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
plt.title("PCA of Novel Lung Shape Radiomic Biomarkers")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,
    columns=["PC1", "PC2"],
    index=X.columns
)

print(loadings.sort_values("PC1", ascending=False))

**Novelty**

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/lung_radiomics_novel.csv")

In [ ]:
df["axis_ratio"] = (
    df["original_shape_MajorAxisLength"] /
    df["original_shape_MinorAxisLength"]
)

df["shape_complexity_index"] = (
    df["original_shape_Elongation"] *
    df["original_shape_Flatness"] *
    df["axis_ratio"]
)

In [ ]:
print(df[["PatientID", "shape_complexity_index"]])

In [ ]:
df.to_csv("/content/lung_radiomics_novel.csv", index=False)
print("✅ shape_complexity_index added and saved")

In [ ]:
median_value = df["shape_complexity_index"].median()

df["Complexity_Group"] = np.where(
    df["shape_complexity_index"] >= median_value,
    "High_Complexity",
    "Low_Complexity"
)

print(df[["PatientID", "shape_complexity_index", "Complexity_Group"]])
print("\nGroup counts:")
print(df["Complexity_Group"].value_counts())

In [ ]:
from scipy.stats import mannwhitneyu

features_to_test = [
    "original_shape_MajorAxisLength",
    "original_shape_MinorAxisLength",
    "original_shape_Elongation",
    "original_shape_Flatness",
    "original_shape_VoxelVolume"
]

results = []

for feat in features_to_test:
    group_high = df[df["Complexity_Group"] == "High_Complexity"][feat]
    group_low  = df[df["Complexity_Group"] == "Low_Complexity"][feat]

    stat, p = mannwhitneyu(group_high, group_low, alternative="two-sided")

    results.append([feat, p])

stats_df = pd.DataFrame(results, columns=["Feature", "p_value"])
print(stats_df)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,5))

df.boxplot(
    column="shape_complexity_index",
    by="Complexity_Group",
    grid=False
)

plt.title("Lung Shape Complexity Stratification")
plt.suptitle("")
plt.ylabel("Shape Complexity Index")
plt.tight_layout()
plt.show()

**Unsupervised clustering (phenotypes)**

In [ ]:
import pandas as pd

df = pd.read_csv("/content/lung_radiomics_novel.csv")
print(df.shape)
df.head()

In [ ]:
features = [
    "original_shape_Elongation",
    "original_shape_Flatness",
    "original_shape_MajorAxisLength",
    "original_shape_MinorAxisLength",
    "original_shape_LeastAxisLength",
    "shape_complexity_index"
]

X = df[features]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertia = []
K = range(2, 6)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(K, inertia, marker='o')
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia")
plt.title("Elbow Method for Lung Phenotypes")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df["phenotype"] = kmeans.fit_predict(X_scaled)

In [ ]:
print(df[["PatientID", "shape_complexity_index", "phenotype"]])

In [ ]:
summary = df.groupby("phenotype")[features].mean()
print(summary)

**Visualization of lung phenotypes**

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce to 2 dimensions
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Plot
plt.figure(figsize=(7,5))
for phenotype in sorted(df["phenotype"].unique()):
    subset = df["phenotype"] == phenotype
    plt.scatter(
        X_pca[subset, 0],
        X_pca[subset, 1],
        label=f"Phenotype {phenotype}",
        s=80
    )

plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("Lung Structural Phenotypes (Shape-Based Radiomics)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(6,5))

for phenotype in sorted(df["phenotype"].unique()):
    subset = df["phenotype"] == phenotype
    plt.scatter(
        df.loc[subset, "original_shape_MajorAxisLength"],
        df.loc[subset, "shape_complexity_index"],
        label=f"Phenotype {phenotype}",
        s=80
    )

plt.xlabel("Major Axis Length")
plt.ylabel("Shape Complexity Index (Novel)")
plt.title("Phenotype Separation Using Novel Shape Feature")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(6,4))
sns.boxplot(
    x="phenotype",
    y="shape_complexity_index",
    data=df
)

plt.xlabel("Phenotype")
plt.ylabel("Shape Complexity Index")
plt.title("Distribution of Novel Shape Complexity Index")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv("/content/lung_radiomics_novel.csv")

plt.figure(figsize=(8, 5))
plt.bar(df["PatientID"], df["shape_complexity_index"])
plt.xticks(rotation=45, ha="right")
plt.xlabel("Patient ID")
plt.ylabel("Shape Complexity Index")
plt.title("Lung Shape Complexity Across Patients")

# 🔹 STEP D4: Save figure
plt.savefig("phenotype_shape_complexity.png",
            dpi=300,
            bbox_inches="tight")

plt.show()

**Statistical validation (effect size + p-values)bold text**

In [ ]:
import pandas as pd

df = pd.read_csv("/content/lung_radiomics_novel.csv")
print(df.shape)
df.head()

In [ ]:
#Create two groups using your novel feature

feature = "shape_complexity_index"

median_val = df[feature].median()

group_low  = df[df[feature] <= median_val][feature]
group_high = df[df[feature] >  median_val][feature]

print("Low group size :", len(group_low))
print("High group size:", len(group_high))

In [ ]:
import numpy as np

def cohens_d(x, y):
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(((nx-1)*x.std()**2 + (ny-1)*y.std()**2) / (nx+ny-2))
    return (x.mean() - y.mean()) / pooled_std

d_value = cohens_d(group_high, group_low)
print("Cohen's d:", d_value)

In [ ]:
from scipy.stats import mannwhitneyu

u_stat, p_value = mannwhitneyu(group_high, group_low, alternative="two-sided")

print("U statistic:", u_stat)
print("p-value:", p_value)

In [ ]:
results = pd.DataFrame({
    "Feature": [feature],
    "Cohens_d": [d_value],
    "p_value": [p_value],
    "Mean_low": [group_low.mean()],
    "Mean_high": [group_high.mean()]
})

results.to_csv("/content/statistical_validation_results.csv", index=False)
results

**Correlation analysis (prove non-redundancy)**

In [ ]:
import pandas as pd

df = pd.read_csv("/content/lung_radiomics_novel.csv")

novel_feature = "shape_complexity_index"

# Select only numeric radiomics (exclude PatientID)
radiomics_features = df.drop(columns=["PatientID"])

print("Total features:", radiomics_features.shape[1])

In [ ]:
corr = radiomics_features.corr(method="spearman")

novel_corr = corr[novel_feature].drop(novel_feature)

novel_corr.sort_values()

In [ ]:
high_corr = novel_corr[abs(novel_corr) > 0.7]

print("Highly correlated features (>0.7):")
high_corr

In [ ]:
mean_abs_corr = abs(novel_corr).mean()
max_abs_corr  = abs(novel_corr).max()

print("Mean |correlation| :", mean_abs_corr)
print("Max  |correlation| :", max_abs_corr)

In [ ]:
novel_corr.to_csv("/content/novel_feature_correlations.csv")

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
novel_corr.sort_values().plot(kind="barh")
plt.title("Correlation of Novel Feature with Radiomics")
plt.xlabel("Spearman Correlation")
plt.tight_layout()
plt.savefig("/content/novel_feature_correlation_plot.png")
plt.show()

**Machine Learning validation 🔥**

In [ ]:
import pandas as pd

df = pd.read_csv("/content/lung_radiomics_novel.csv")

print(df.shape)
df.head()

In [ ]:
import numpy as np

# REMOVE this once real labels are available
np.random.seed(42)
df["label"] = np.random.randint(0, 2, size=len(df))

In [ ]:
from sklearn.preprocessing import StandardScaler

X_radiomics = df.drop(columns=["PatientID", "label", "shape_complexity_index"])
X_novel     = df[["shape_complexity_index"]]
X_combined  = df.drop(columns=["PatientID", "label"])

y = df["label"]

scaler = StandardScaler()
X_radiomics = scaler.fit_transform(X_radiomics)
X_novel     = scaler.fit_transform(X_novel)
X_combined  = scaler.fit_transform(X_combined)

In [ ]:
# Model + LOOCV

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut, cross_val_score

loo = LeaveOneOut()
model = LogisticRegression(max_iter=1000)

In [ ]:
# ⚠️ Exploratory results – not used for final evaluation

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score

y_pred_prob_rad = cross_val_predict(
    model, X_radiomics, y, cv=loo, method="predict_proba"
)[:,1]

y_pred_prob_com = cross_val_predict(
    model, X_combined, y, cv=loo, method="predict_proba"
)[:,1]

print("AUC Radiomics:", roc_auc_score(y, y_pred_prob_rad))
print("AUC Combined:", roc_auc_score(y, y_pred_prob_com))

In [ ]:
# ⚠️ Exploratory results – not used for final evaluation

acc_rad = cross_val_score(model, X_radiomics, y, cv=loo, scoring="accuracy")
acc_com = cross_val_score(model, X_combined,  y, cv=loo, scoring="accuracy")

print("Radiomics only accuracy:", acc_rad.mean())
print("Radiomics + Novel accuracy:", acc_com.mean())

In [ ]:
df_numeric = df.select_dtypes(include=["number"])

In [ ]:
df_numeric.corr()["shape_complexity_index"].sort_values(ascending=False)

In [ ]:
corr = df_numeric.corr()["shape_complexity_index"]
corr = corr.drop("shape_complexity_index")
corr.sort_values(ascending=False)

In [ ]:
# Feature stability
df_numeric.std().sort_values(ascending=False)

In [ ]:
# Axis Disparity Index (ADI)

# ADI=Lmajor​Lmajor​−Lleast​​

df["axis_disparity_index"] = (
    df["original_shape_MajorAxisLength"] -
    df["original_shape_LeastAxisLength"]
) / df["original_shape_MajorAxisLength"]

In [ ]:
df["morphology_heterogeneity_score"] = (
    df["shape_complexity_index"] *
    df["axis_disparity_index"]
)

In [ ]:
# Boxplot by label
df.boxplot(column="shape_complexity_index", by="label")

In [ ]:
from scipy.stats import wilcoxon

stat, p = wilcoxon(acc_com, acc_rad)

print("Wilcoxon p-value:", p)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.boxplot([acc_rad, acc_com], labels=["Radiomics", "Radiomics + Novel"])
plt.ylabel("Accuracy")
plt.title("Impact of Novel Feature on Classification Performance")
plt.tight_layout()
plt.savefig("/content/novel_feature_ml_comparison.png")
plt.show()

**Validate correlations of ADI with label**

In [ ]:
numeric_df = df.select_dtypes(include=["number"])

In [ ]:
#Correlation with label
numeric_df[["label", "shape_complexity_index", "axis_disparity_index"]].corr()

In [ ]:
# Check redundancy (SCI vs ADI)
numeric_df[["shape_complexity_index", "axis_disparity_index"]].corr()

In [ ]:
# Rank correlation against all features
numeric_df.corr()["axis_disparity_index"].sort_values(ascending=False)

**Fix evaluation properly (this is CRITICAL)**

**Leave-One-Out Cross Validation (LOOCV)**

In [ ]:
# Prepare data correctly
import pandas as pd
import numpy as np

from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
labels = {
    "TCGA-34-5241": 1,
    "TCGA-34-8455": 0,
    "TCGA-60-2695": 1,
    "TCGA-60-2696": 0,
    "TCGA-60-2710": 1,
    "TCGA-60-2711": 0,
    "TCGA-60-2716": 1,
    "TCGA-60-2721": 0
}

df["label"] = df["PatientID"].map(labels)

In [ ]:
print(df[["PatientID", "label"]])

In [ ]:
df = pd.read_csv("/content/lung_radiomics_novel.csv")

labels = {
    "TCGA-34-5241": 1,
    "TCGA-34-8455": 0,
    "TCGA-60-2695": 1,
    "TCGA-60-2696": 0,
    "TCGA-60-2710": 1,
    "TCGA-60-2711": 0,
    "TCGA-60-2716": 1,
    "TCGA-60-2721": 0
}

df["label"] = df["PatientID"].map(labels)

In [ ]:
print(df.columns.tolist())

In [ ]:
X_radiomics = df.drop(columns=["PatientID", "label"], errors="ignore")

X_radiomics_only = X_radiomics.drop(
    columns=["shape_complexity_index", "axis_disparity_index"],
    errors="ignore"
)

X_combined = X_radiomics
y = df["label"].values

In [ ]:
print(df.columns.tolist())

In [ ]:
required_cols = ["PatientID", "label"]
missing = [c for c in required_cols if c not in df.columns]
print("Missing columns:", missing)

In [ ]:
#Define LOOCV evaluation function
def loocv_evaluate(X, y):
    loo = LeaveOneOut()
    y_true, y_pred, y_prob = [], [], []

    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = Pipeline([
            ("scaler", StandardScaler()),
            ("clf", LogisticRegression(max_iter=1000))
        ])

        model.fit(X_train, y_train)

        y_true.append(y_test[0])
        y_pred.append(model.predict(X_test)[0])
        y_prob.append(model.predict_proba(X_test)[0, 1])

    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)

    return acc, auc

In [ ]:
# Run proper evaluation
acc_rad, auc_rad = loocv_evaluate(X_radiomics_only, y)
acc_comb, auc_comb = loocv_evaluate(X_combined, y)

print(f"LOOCV Radiomics Accuracy: {acc_rad:.3f}")
print(f"LOOCV Radiomics AUC:      {auc_rad:.3f}\n")

print(f"LOOCV Combined Accuracy:  {acc_comb:.3f}")
print(f"LOOCV Combined AUC:       {auc_comb:.3f}")